In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-31d5ee5c-d94b-9a84-67a0-f95a559a85ed)


In [2]:
pip install --upgrade efficientnet-pytorch

  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=6ff513dd77f9db09baaeca4a32e30c3668218458bd01faadee001c26002ff769
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.3/447.3 MB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import re
import csv

import torch
import torch.nn as nn
import torchmetrics
import torchvision
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import nvidia.dali.ops as ops
import nvidia.dali.types as types
from nvidia.dali.pipeline import Pipeline
from nvidia.dali.plugin.pytorch import DALIClassificationIterator, LastBatchPolicy

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
NUM_CLASSES = 200
BATCH_SIZE = 40
NUM_THREADS = 8
NUM_EPOCHS = 14
VAL_SIZE = .1
NET_NAME = 'efficientnet-b3'

TRAIN_DIR = '/kaggle/input/bird11/train'
TEST_DIR = '/kaggle/input/bird11/test'

In [7]:
IMAGES = []
IMAGES_TARGETS = []

for dirname,_, filenames in os.walk(TRAIN_DIR):
    m = re.search(r'(?<=/)\d+', dirname)         #get labels with regular expression
    
    for filename in filenames:
        if filename.endswith('.jpg'):
            IMAGES.append((os.path.join(dirname, filename), int(m.group(0))))
            IMAGES_TARGETS.append(int(m.group(0)))
            
TRAIN_IMAGES, VAL_IMAGES = train_test_split(IMAGES,test_size=VAL_SIZE,
                                            shuffle=True,
                                            stratify=IMAGES_TARGETS)

train_dirs, train_labels = zip(*TRAIN_IMAGES)
val_dirs, val_labels = zip(*VAL_IMAGES)

In [8]:
train_labels

(113,
 174,
 157,
 9,
 24,
 114,
 1,
 141,
 192,
 79,
 153,
 78,
 56,
 20,
 77,
 124,
 128,
 71,
 5,
 97,
 52,
 53,
 198,
 13,
 16,
 173,
 59,
 113,
 119,
 132,
 48,
 155,
 151,
 146,
 11,
 26,
 51,
 10,
 193,
 102,
 121,
 179,
 5,
 25,
 143,
 17,
 55,
 13,
 3,
 95,
 45,
 106,
 98,
 142,
 189,
 48,
 64,
 191,
 77,
 70,
 26,
 170,
 0,
 174,
 149,
 17,
 96,
 0,
 23,
 131,
 169,
 69,
 47,
 13,
 58,
 45,
 88,
 137,
 81,
 68,
 157,
 90,
 95,
 156,
 179,
 62,
 70,
 110,
 36,
 177,
 177,
 156,
 73,
 172,
 112,
 33,
 27,
 165,
 136,
 98,
 117,
 81,
 126,
 42,
 48,
 149,
 66,
 186,
 76,
 69,
 34,
 9,
 171,
 101,
 109,
 145,
 120,
 194,
 130,
 105,
 195,
 97,
 142,
 165,
 126,
 26,
 75,
 21,
 75,
 28,
 26,
 46,
 99,
 37,
 45,
 133,
 129,
 100,
 76,
 92,
 72,
 171,
 184,
 193,
 0,
 125,
 35,
 175,
 130,
 170,
 66,
 75,
 147,
 169,
 36,
 45,
 89,
 60,
 36,
 125,
 150,
 79,
 102,
 93,
 130,
 1,
 98,
 39,
 100,
 140,
 183,
 130,
 199,
 163,
 53,
 58,
 163,
 143,
 81,
 16,
 156,
 35,
 106,
 25,
 65,


In [9]:
class MyTrainPipe(Pipeline):
    def __init__(self, files, labels, batch_size, num_threads, device_id):
        super(MyTrainPipe, self).__init__(batch_size, num_threads, device_id, seed=12)
        self.input = ops.readers.File(files=files, labels=labels, random_shuffle=True)
        self.decode = ops.decoders.Image(device="mixed", output_type=types.RGB)
        self.res = ops.RandomResizedCrop(device="gpu", size=(300, 300), random_area=[0.08, 1.25])
        self.cmn = ops.CropMirrorNormalize(device="gpu",
                                            dtype=types.FLOAT,
                                            output_layout=types.NCHW,
                                            mean=[0.485 * 255, 0.456 * 255, 0.406 * 255],
                                            std=[0.229 * 255, 0.224 * 255, 0.225 * 255])
        self.coin = ops.random.CoinFlip(probability=0.5)

    def define_graph(self):
        self.jpegs, self.labels = self.input(name="Reader")
        images = self.decode(self.jpegs)
        images = self.res(images)
        output = self.cmn(images, mirror=self.coin())
        return output, self.labels


class MyValPipe(Pipeline):
    def __init__(self, files, labels, batch_size, num_threads, device_id):
        super(MyValPipe, self).__init__(batch_size, num_threads, device_id, seed=12)
        self.input = ops.readers.File(files=files, labels=labels)
        self.decode = ops.decoders.Image(device="mixed", output_type=types.RGB)
        self.res = ops.Resize(device="gpu", resize_shorter=324, interp_type=types.INTERP_TRIANGULAR)
        self.cmn = ops.CropMirrorNormalize(device="gpu",
                                            dtype=types.FLOAT,
                                            output_layout=types.NCHW,
                                            crop=(300, 300),
                                            mean=[0.485 * 255, 0.456 * 255, 0.406 * 255],
                                            std=[0.229 * 255, 0.224 * 255, 0.225 * 255])

    def define_graph(self):
        self.jpegs, self.labels = self.input(name="Reader")
        images = self.decode(self.jpegs)
        images = self.res(images)
        output = self.cmn(images)
        return output, self.labels


def _make_net_iterator(type, files, labels, batch_size, num_threads, device_id=0):
    if type == 'train':
        ppl = MyTrainPipe(files=files, labels=labels, batch_size=batch_size, 
                                num_threads=num_threads, device_id=device_id)
        ppl.build()
        net_iter = DALIClassificationIterator(ppl, 
                                                     reader_name="Reader",
                                                     last_batch_policy=LastBatchPolicy.PARTIAL,
                                                     auto_reset=True)
        return net_iter
    
    elif type == 'val':
        ppl = MyValPipe(files=files, labels=labels, batch_size=batch_size, 
                            num_threads=num_threads, device_id=device_id)
        ppl.build()
        net_iter = DALIClassificationIterator(ppl, reader_name="Reader",
                                                   last_batch_policy=LastBatchPolicy.PARTIAL,
                                                   auto_reset=True)
        return net_iter

In [10]:
class MyNetwork(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.net = EfficientNet.from_pretrained(NET_NAME) 
        self.criterion = torch.nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy()
        self.lr = 1e-3
        
    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        x = abs(batch[0]["data"].squeeze().float())
        y = abs(batch[0]["label"].squeeze().long())
        y_hat = self.net(x)
        acc = self.accuracy(y_hat, y)
        self.log('train_acc', acc, prog_bar=True)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss        
   
    def validation_step(self, batch, batch_idx):
        x = batch[0]["data"].squeeze().float()
        y = batch[0]["label"].squeeze().long()
        y_hat = self.net(x)
        acc = self.accuracy(y_hat, y)
        self.log('val_acc', acc, prog_bar=True, logger=True)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss, prog_bar=True, logger=True)

    def configure_optimizers(self):
        return torch.optim.SGD(self.net.parameters(), lr=self.lr, momentum=.9)

In [11]:
train_dl = _make_net_iterator(type='train', 
                                  files=train_dirs, 
                                  labels=train_labels,
                                  batch_size=BATCH_SIZE, 
                                  num_threads=NUM_THREADS)

val_dl = _make_net_iterator(type='val', 
                                files=val_dirs, 
                                labels=val_labels,
                                batch_size=BATCH_SIZE, 
                                num_threads=NUM_THREADS)

net = MyNetwork()

net_checkpoint = ModelCheckpoint(monitor = "val_acc",
                                 mode='max',
                                 verbose=True)

trainer = pl.Trainer(gpus=1,
                     max_epochs=NUM_EPOCHS,
                     accelerator='gpu',
                     callbacks=[net_checkpoint],
        )
trainer.tune(net, train_dl, val_dl)
trainer.fit(net, train_dl, val_dl)

trainer.save_checkpoint("best_net.ckpt")

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [12]:
test_transform = transforms.Compose([transforms.Resize(324),
                                     transforms.CenterCrop(300),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

net = MyNetwork.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
net.eval()
net.to(device)

headers = ['image_name', 'class']
results = []

for filename in tqdm(os.listdir(TEST_DIR)):
    if filename.endswith(".jpg"):
        img = Image.open(TEST_DIR+ '/' + filename).convert("RGB")
        data = test_transform(img)
        data = data.to(device)
        data = data.float()
        data = data.unsqueeze(0)
        pred = net(data)
        output = pred.max(1)[1]
        results.append({'image_name':filename, 'class':output.item()})

Loaded pretrained weights for efficientnet-b3


100%|██████████| 2000/2000 [01:06<00:00, 29.99it/s]


In [13]:
with open('submission.csv','w',newline='') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(results)